In [ ]:
import cv2
import numpy as np

In [ ]:
# function to capture the video

cap = cv2.VideoCapture(0)

# Parameters for ShiTomasi corner detection
feature_params = dict( maxCorners=100,
                       qualityLevel=0.3,
                       minDistance=7,
                       blockSize=7)

# Parameters for lucas kanade optical flow
lucas_params = dict( winSize=(15, 15),
                  maxLevel=2,
                  criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# color creation
color = np.random.randint(0, 255, (100, 3))

# Taking the first frame and finding corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Creating a mask image for drawing the lines if movement is on left or right
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lucas_params)

    # Point selection
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        if a > c:
            print('LEFT')
        elif c > a:
            print('RIGHT')
        else:
            print('SAME')
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)

    img = cv2.add(frame, mask)

    # Function for showing image
    cv2.imshow('frame', img)

    
    k = cv2.waitKey(30) & 0xff
    if k == 32:
        break

    # updation of previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)


cap.release()
cv2.destroyAllWindows()
